In [46]:
from flaml import AutoML
import pandas as pd
import sklearn.metrics
from sklearn.model_selection import train_test_split
# Initialize an AutoML instance
automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 3600,
    "metric": 'RMSE',
    "task": 'regression',
    "log_file_name": "energia.log",
}
data = pd.read_excel('BASEi.xlsx', index_col=0)
df_dummies = pd.get_dummies(data['Mes'])
df_dummies.columns = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
data = data.join(df_dummies)
#Split data into train and test the X is all the columns minus 'Energia GBA' which is the target
X = data.drop(['Energia GBA', 'Energia PBA', 'lag GBA', 'Año', 'Mes'], axis=1)
y = data['Energia PBA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X.tail()

,Hora,Clima PBA,Clima GBA,lag PBA,Laboral,Semi Laboral,Feriado o domingo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
32851,19,16.8,16.8,1939.2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
32852,20,16.9,16.9,2115.4,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
32853,21,16.9,16.9,2164.6,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
32854,22,16.9,16.9,2093.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
32855,23,16.4,16.4,2061.2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [47]:
# Train with labeled input data
automl.fit(X_train=X_train, y_train=y_train, early_stop=True,
           **automl_settings)

[flaml.automl: 11-23 20:47:29] {1485} INFO - Data split method: uniform
[flaml.automl: 11-23 20:47:29] {1489} INFO - Evaluation method: cv
[flaml.automl: 11-23 20:47:29] {1540} INFO - Minimizing error metric: RMSE
[flaml.automl: 11-23 20:47:29] {1577} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree']
[flaml.automl: 11-23 20:47:29] {1826} INFO - iteration 0, current learner lgbm
[flaml.automl: 11-23 20:47:29] {1944} INFO - Estimated sufficient time budget=1624s. Estimated necessary time budget=2s.
[flaml.automl: 11-23 20:47:29] {2029} INFO -  at 0.2s,	estimator lgbm's best error=192.4945,	best estimator lgbm's best error=192.4945
[flaml.automl: 11-23 20:47:29] {1826} INFO - iteration 1, current learner lgbm
[flaml.automl: 11-23 20:47:30] {2029} INFO -  at 0.4s,	estimator lgbm's best error=192.4945,	best estimator lgbm's best error=192.4945
[flaml.automl: 11-23 20:47:30] {1826} INFO - iteration 2, current learner lgbm
[flaml.automl: 11-23 20:47:30] {2029} 

In [48]:
# Print the best model
print(automl.model.estimator)

In [49]:
#Predict the test set
predictions = automl.predict(X_test)
#Print the mean absolute error
print("Mean absolute error: %.2f" % sklearn.metrics.mean_absolute_error(y_test, predictions))

Mean absolute error: 60.93


In [50]:
#Import xlsx
data_oct = pd.read_excel('oct.xlsx', index_col=0)
#Split data into train and test the X is all the columns minus 'Energia GBA' which is the target
X_oct = data_oct.drop(['Energia GBA', 'Energia PBA', 'lag GBA', 'Tipo Dia', 'Año', 'Mes'], axis=1)
y_oct = data_oct['Energia PBA']
#Predict the test set
predictions = automl.predict(X_oct)

#Print the mean absolute error
print("RMSE: %.2f" % sklearn.metrics.mean_squared_error(y_oct, predictions, squared = False))
print("MSE: %.2f" % sklearn.metrics.mean_squared_error(y_oct, predictions))
print("MAE: %.2f" % sklearn.metrics.mean_absolute_error(y_oct, predictions))
predictions = pd.DataFrame(predictions)
#Export predictions
predictions.to_csv('predictions.csv')

RMSE: 133.75
MSE: 17890.31
MAE: 105.86
